<a href="https://colab.research.google.com/github/eladoni1/Detectron2-Shenanigans/blob/main/Detectron_Shenanigans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Deleting everything unrelated / from previous runs
import shutil
import os
for filename in os.listdir('./'):
  shutil.rmtree(filename)

In [ ]:
# installing requirements for Detectron2
!python -m pip install pyyaml==5.1
import sys, os, distutils.core

!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 274 kB 7.3 MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp38-cp38-linux_x86_64.whl size=44089 sha256=2d874ef88d86c68bfc957f0007eeb6c11062f9b8427d96837b03b29f31f2b817
  Stored in directory: /root/.cache/pip/wheels/52/dd/2b/10ff8b0ac81b93946bb5fb9e6749bae2dac246506c8774e6cf
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask 2022.2.1 requires pyyaml>=5.3.1, but you have pyyaml 5.1 which is incompatible.
Cloning into 'detectron2'...
remote: Enumerating objects: 14718, done.
remote: Total 14718 (delta 0), reused 0 (delta 0), pack-reused 14718
Receivi

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


['Pillow>=7.1', 'matplotlib', 'pycocotools>=2.0.2', 'termcolor>=1.1', 'yacs>=0.1.8', 'tabulate', 'cloudpickle', 'tqdm>4.29.0', 'tensorboard', 'fvcore<0.1.6,>=0.1.5', 'iopath<0.1.10,>=0.1.7', 'omegaconf>=2.1', 'hydra-core>=1.1', 'black', 'timm', 'packaging']
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 3.9 MB/s 
     |████████████████████████████████| 79 kB 8.1 MB/s 
     |████████████████████████████████| 154 kB 65.9 MB/s 
     |████████████████████████████████| 1.5 MB 48.0 MB/s 
     |████████████████████████████████| 549 kB 30.5 MB/s 
     |████████████████████████████████| 117 kB 67.2 MB/s 
     |████████████████████████████████| 96 kB 4.6 MB/s 
     |████████████████████████████████| 182 kB 75.0 MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61431 sha256=ab6def8d4ad5781c973bde862690ef3657bb7c490212db90111a29b7ecc3d351
  Stored in directory:

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import torch, detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
# Pre-trained models running panoptic detection (object detection - 'things', and then after ignoring the objects, detecting the 'stuff', roads trees sea sky)
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml")
# Changed to CPU (GPU and TPU sessions are hard to get..)
cfg.MODEL.DEVICE = 'cpu'
predictor = DefaultPredictor(cfg)


[12/24 11:09:53 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/COCO-PanopticSegmentation/panoptic_fpn_R_101_3x/139514519/model_final_cafdb1.pkl ...


model_final_cafdb1.pkl: 261MB [00:03, 68.5MB/s]                           


In [ ]:
!python -m pip install wget
import wget
import zipfile
from google.colab import files

# Allowing user to upload dataset / downloading it from another source.
yes_or_no = ''

while yes_or_no != 'y' and yes_or_no != 'n':
    yes_or_no = input('\nDO YOU HAVE YOUR DATASET LOCALLY ON YOUR PC? (ZIP ONLY) y/n : ')
    yes_or_no = yes_or_no.lower()
if yes_or_no == 'y':
  uploaded = files.upload()
  file_name = next(iter(uploaded))
  with zipfile.ZipFile('./' + file_name, 'r') as zip_ref:
      zip_ref.extractall('./extracted_data/')
  path = './extracted_data/' + os.path.splitext(file_name)[0]
else:
  yes_or_no = ''
  while yes_or_no != 'y' and yes_or_no != 'n':
    yes_or_no = input('\nDO YOU HAVE YOUR DATASET LOCALLY ON THIS RUNTIME? (UNZIPPED) y/n : ')
    yes_or_no = yes_or_no.lower()
  if yes_or_no == 'y':
    path = input('\nINSERT LOCAL PATH:\n')
    while not os.path.exists(path):
      path = input('\nPATH DO NOT EXIST.\nINSERT LOCAL PATH:\n')
  else:
    have_wget = ''
    while have_wget.lower() != 'y' and have_wget.lower() != 'n':
      have_wget = input('DO YOU HAVE URL (DATASET URL)? y/n : ')
    if have_wget == 'n':
      url = "http://images.cocodataset.org/zips/train2017.zip"
    else:
      url = input('ENTER WGET URL (DATASET URL):\n  ')
    file_name = wget.download(url)
    with zipfile.ZipFile('./' + file_name, 'r') as zip_ref:
      zip_ref.extractall('./extracted_data/')
    path = './extracted_data/' + os.path.splitext(file_name)[0]
    os.remove('./' + file_name) # free some space - deletes zip file

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

DO YOU HAVE YOUR DATASET LOCALLY ON YOUR PC? (ZIP ONLY) y/n : y


Saving questions.zip to questions (1).zip


In [ ]:
if not os.path.exists('./cropped_images'):
    os.makedirs('./cropped_images')

# Classifier ID for 'sky'
sky_idx = MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).stuff_classes.index('sky')

# For each file in the directory..
for filename in os.listdir(path):
  # Check for image extension
  if not filename.lower().endswith(('.png', '.jpg', '.jpeg')):
    continue
  im = cv2.imread(path + '/' + filename)
  
  # Running the predictor
  panoptic_seg, segments_info = predictor(im)["panoptic_seg"]

  # Sky can be divded into segments, therefore we will collect these under one list
  boxes_idx = []

  # If it's not a 'thing' [stuff, surroundings, helps us get rid of object detection]
  # And if the category_id is equal to the Classifier ID - meaning it's classified as sky.
  for item_idx, item in enumerate(segments_info):
    if not item['isthing'] and item['category_id'] == sky_idx:
      boxes_idx.append(item['id'])

  # Starting from the first row (highest pixel row)
  maximum_row = 0

  # Find where sky is no longer detected
  for id in boxes_idx:
    for row_idx, row in enumerate(panoptic_seg):
      if id in row:
        continue
      maximum_row = max(row_idx, maximum_row)
      break
  
  # Crop the picture and save it
  im2 = im[maximum_row:, :]
  try:
    if maximum_row != im.shape[1]:
      cv2.imwrite("cropped_images/" + 'cropped_' + filename, im2)
  except:
    print("FAILED:", filename)

  # Useable when using big datasets - deletes the file from the dataset's folder.
  # os.remove(path + '/' + filename)


/usr/local/lib/python3.8/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
